# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [107]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [108]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

r =  requests.get(url)
r.status_code

200

In [109]:
data_json = r.json()
df_json = pd.DataFrame(data_json['items'])
df_json.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10108,Município,São Pedro da Aldeia,3305208,RJ,None,Arquivado,17944.001122/2008-21,2009-03-06T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,598900.0,0,0,13/03/2009


In [110]:
#2) seu Código
df_json.status.value_counts()

Deferido                                                 101
Arquivado                                                 53
Regularizado                                              26
Arquivado por decurso de prazo                            24
Deferido (PVL-IF)                                         22
Encaminhado à PGFN com manifestação técnica favorável     14
Arquivado a pedido                                         6
Em retificação pelo interessado                            4
Em análise                                                 3
Arquivado pela STN                                         1
Em retificação pelo interessado (PVL-IF)                   1
Name: status, dtype: int64

In [111]:
# 3) Seu código aqui
ano = []
data = df_json['data_status'].str.split(pat = '/')
for date in data:
    try:
        ano.append(date[2])
    except:
        ano.append(np.nan)
df_json['ano'] = ano
df_json.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,ano
0,10108,Município,São Pedro da Aldeia,3305208,RJ,None,Arquivado,17944.001122/2008-21,2009-03-06T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,598900.0,0,0,13/03/2009,2009


In [112]:
df_json.ano.value_counts()

2008    37
2007    29
2023    24
2014    20
2010    19
2019    17
2020    15
2013    15
2012    13
2022    11
2011    10
2009     9
2015     8
2021     5
2017     5
2002     4
2018     4
2006     4
2016     4
2004     1
2003     1
Name: ano, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [113]:
#1) Seu código aqui

def consulta(UF,tipo_interessado):
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

    parametros = {
        'uf': UF,
        'tipo_interessado': tipo_interessado
    }
    try:
        r = requests.get(url, params=parametros)
        r.status_code  
        data = r.json()  
    
        df_json = pd.DataFrame(data['items'])
    
        return df_json
    except Error as erro:
        print('Erro de conexão: ', erro)

In [114]:
consulta('AC', 'Estado').head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,8789,Estado,Acre,12,AC,None,Deferido,17944.000841/2013-92,2014-03-25T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,34150000.0,1,0,07/04/2014


In [115]:
# 2) Seu código aqui
consulta('MG','Estado').status.value_counts()

Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Arquivado por decurso de prazo                            1
Indeferido                                                1
Name: status, dtype: int64

In [116]:
mg[mg['status'] == 'Arquivado por decurso de prazo'].status.value_counts()

# mostrando as linhas com o status:
mg[mg['status'] == 'Arquivado por decurso de prazo']

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
19,20534,Estado,Minas Gerais,31,MG,00000.000000/2017-69,Arquivado por decurso de prazo,17944.001113/2015-60,2017-07-26T03:00:00Z,Operação contratual interna,PMAE,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,7668000.0,0,0,16/03/2018


In [117]:
# 3) Seu código códig

ba = consulta('BA', 'Município')
status_ba = ba[ba.status == 'Deferido']
status_ba['interessado'].value_counts()


Luís Eduardo Magalhães    16
Vitória da Conquista      12
Barreiras                 10
Lauro de Freitas          10
Camaçari                  10
                          ..
Xique-Xique                1
Paulo Afonso               1
Itaguaçu da Bahia          1
Ibotirama                  1
Conceição do Jacuípe       1
Name: interessado, Length: 182, dtype: int64

In [118]:
# 4) Seu código aqui
consulta('BA', 'Município').to_csv('c:/users/datas/Downloads/CSV/solicitacoes_bahia.csv', sep =',', index = False, mode = 'w')